In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

In [ ]:
# Define Helper Functions
# 1. Split the data
# Split a dataset based on a feature and a given threshold
def split_data (feature_index, threshold, data):
    left, right = [], []
    for row in data:
        if row[feature_index] <= threshold:
            left.append(row)
        else:
            right.append(row)
    return left, right

# 2. Gini Impurity 
# weighted gini impurity = (left group size / total samples)* Gini_left + (right group size / total samples)* Gini_right
def gini_impurity(groups, classes, class_index):
    '''
    Parameters:
        groups: A list of all subsets (left and right groups)
        classes: Target classes, e.g. 0 & 1, male and female
        class_index: Column index of class label in the data
    '''
    total_sample = sum(len(group) for group in groups) 
    weighted_gini = 0
    for group in groups:
        group_size = len(group)
        if group_size == 0:
            continue # skip empty groups
        
        gini = 0
        for class_label in classes:
            class_size = sum(1 for row in group if row[class_index] == class_label)
            p = class_size/group_size 
            gini += p**2
        group_gini = 1-gini
        weighted_gini += group_gini * (group_size/total_sample)

    return weighted_gini

In [ ]:
# Find the best split
def find_best_split(data, target_index):
    '''
    Parameters:
        data: Input dataset where rows are samples and columns are features.
        target_index: The index of the column which is chosen to be the target variable.
    
    Returns:
        best_feature_index: The index of the best feature of splitting.
        best_threshold: The threshold or values that gives the best split.
        best_score: The best (lowest) gini impurity score for the split.
    '''
    best_feature_index = None
    best_threshold = None
    best_score = float('inf')

    data = np.array(data)
    n_features = data.shape[1] # number of columns in the data

    # Extract unique class labels from the target column
    classes = np.unique(data[:, target_index])

    for feature_index in range(n_features):
        if feature_index == target_index:
            continue # skip the target column

        # Extract unique values for each feature to test as splitting thresholds:
        thresholds = np.unique(data[:, feature_index])
        for threshold in thresholds:
            # Split the dataset into left and right groups using the defined function
            left_group, right_group = split_data(feature_index, threshold, data)
            groups = [left_group, right_group]

            if len(left_group) == 0 or len(right_group) == 0:
                continue # skip empty groups 

            # Compute gini impurity using the defined function
            score = gini_impurity(groups, classes, target_index)

            # Update the best split if the current score is better (lower)
            if score < best_score:
                best_score = score
                best_feature_index = feature_index
                best_threshold = threshold
                
    return best_feature_index, best_threshold, best_score

In [ ]:
# Build a tree
class DecisionTree:
    def __init__(self, min_sample=2, max_depth=None):
        self.min_sample = min_sample
        self.max_depth = max_depth
        self.tree = None

    def leaf_node(self, data, target_index):
        target_val = [row[target_index] for row in data]
        # Find the most frequent element (class label) in the list
        return max(set(target_val), key = target_val.count)
    
    def recursive_splitting(self, data, target_index, depth = 0):
        # Define stopping criteria 
        if depth >= self.max_depth or len(data)< self.min_sample:
            return self.leaf_node(data, target_index)

        # Find the best split
        best_feature_index, best_threshold, best_score = find_best_split(data, target_index)
        if best_feature_index is None:
            return self.leaf_node(data, target_index)

        # Split the data
        left, right = split_data (best_feature_index, best_threshold, data)
        if not left or not right: # empty groups
            return self.leaf_node(data, target_index)

        # Initiate the recursive process to build the right and left trees respectively 
        left_tree = self.recursive_splitting(left, target_index, depth + 1)
        right_tree = self.recursive_splitting(right, target_index, depth + 1)

        # Return the current node
        return {
            'feature': best_feature_index,
            'threshold': best_threshold,
            'left': left_tree,
            'right': right_tree
        }
    
    def fit(self, data, target_index):
        self.tree = self.recursive_splitting(data, target_index)
    
    def predit(self, row):
        '''
        While not a leaf, evaluate the feature against the threshold 
        and update the node either to the left or right subtree.
        '''
        node = self.tree # initiate a root node
        while isinstance(node, dict):  
            if row[node['feature']] <= node['threshold']:
                node = node['left']
            else:
                node = node['right']
        return node